# Car Price Prediction - Geely Auto

###VAC2311 - Assignment 1

> Linear Regression using PySpark
>
> Ojas Patil - 21BAI1106
>
> Link - https://www.kaggle.com/code/tanmoyx/car-price-prediction-geely-auto/input

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3b45e79c595a32939e2f9321c6eec8e8f49ee15941b8ac7d8fdcc3c84319a5d0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
    import pyspark
    from pyspark.sql import SparkSession

    spark=SparkSession.builder.appName('CarPricePrediction').getOrCreate()

In [3]:
spark

In [5]:
df=spark.read.csv('/content/drive/MyDrive/pyspark/CarPrice_Geely.csv',header=True,inferSchema=True)
df.show()

+------+---------+--------------------+--------+----------+----------+-----------+----------+--------------+---------+---------+--------+---------+----------+----------+--------------+----------+----------+---------+------+----------------+----------+-------+-------+----------+---------+
|car_ID|symboling|             CarName|fueltype|aspiration|doornumber|    carbody|drivewheel|enginelocation|wheelbase|carlength|carwidth|carheight|curbweight|enginetype|cylindernumber|enginesize|fuelsystem|boreratio|stroke|compressionratio|horsepower|peakrpm|citympg|highwaympg|    price|
+------+---------+--------------------+--------+----------+----------+-----------+----------+--------------+---------+---------+--------+---------+----------+----------+--------------+----------+----------+---------+------+----------------+----------+-------+-------+----------+---------+
|     1|        3|  alfa-romero giulia|     gas|       std|       two|convertible|       rwd|         front|     88.6|    168.8|    6

In [6]:
df.printSchema()

root
 |-- car_ID: integer (nullable = true)
 |-- symboling: integer (nullable = true)
 |-- CarName: string (nullable = true)
 |-- fueltype: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- doornumber: string (nullable = true)
 |-- carbody: string (nullable = true)
 |-- drivewheel: string (nullable = true)
 |-- enginelocation: string (nullable = true)
 |-- wheelbase: double (nullable = true)
 |-- carlength: double (nullable = true)
 |-- carwidth: double (nullable = true)
 |-- carheight: double (nullable = true)
 |-- curbweight: integer (nullable = true)
 |-- enginetype: string (nullable = true)
 |-- cylindernumber: string (nullable = true)
 |-- enginesize: integer (nullable = true)
 |-- fuelsystem: string (nullable = true)
 |-- boreratio: double (nullable = true)
 |-- stroke: double (nullable = true)
 |-- compressionratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- peakrpm: integer (nullable = true)
 |-- citympg: integer (nullable = t

In [7]:
df.columns

['car_ID',
 'symboling',
 'CarName',
 'fueltype',
 'aspiration',
 'doornumber',
 'carbody',
 'drivewheel',
 'enginelocation',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginetype',
 'cylindernumber',
 'enginesize',
 'fuelsystem',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg',
 'price']

The car ID is simpy for indexing, so we wont consider it.
The Car Names maybe useful, but for now we wont consider it.

All the columns with categorical values will be converted into numerical values.

In [9]:
from pyspark.ml.feature import StringIndexer

inp_cols = ["fueltype", "aspiration", "doornumber", "carbody", "drivewheel", "enginelocation", "enginetype", "cylindernumber", "fuelsystem"]
op_cols = ["fueltype_indexed", "aspiration_indexed", "doornumber_indexed", "carbody_indexed", "drivewheel_indexed", "enginelocation_indexed", "enginetype_indexed", "cylindernumber_indexed", "fuelsystem_indexed"]


indexer = StringIndexer(inputCols=inp_cols,outputCols=op_cols)
df_indexed = indexer.fit(df).transform(df)
df_indexed.show()

+------+---------+--------------------+--------+----------+----------+-----------+----------+--------------+---------+---------+--------+---------+----------+----------+--------------+----------+----------+---------+------+----------------+----------+-------+-------+----------+---------+----------------+------------------+------------------+---------------+------------------+----------------------+------------------+----------------------+------------------+
|car_ID|symboling|             CarName|fueltype|aspiration|doornumber|    carbody|drivewheel|enginelocation|wheelbase|carlength|carwidth|carheight|curbweight|enginetype|cylindernumber|enginesize|fuelsystem|boreratio|stroke|compressionratio|horsepower|peakrpm|citympg|highwaympg|    price|fueltype_indexed|aspiration_indexed|doornumber_indexed|carbody_indexed|drivewheel_indexed|enginelocation_indexed|enginetype_indexed|cylindernumber_indexed|fuelsystem_indexed|
+------+---------+--------------------+--------+----------+----------+----

In [10]:
df_indexed.columns

['car_ID',
 'symboling',
 'CarName',
 'fueltype',
 'aspiration',
 'doornumber',
 'carbody',
 'drivewheel',
 'enginelocation',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginetype',
 'cylindernumber',
 'enginesize',
 'fuelsystem',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg',
 'price',
 'fueltype_indexed',
 'aspiration_indexed',
 'doornumber_indexed',
 'carbody_indexed',
 'drivewheel_indexed',
 'enginelocation_indexed',
 'enginetype_indexed',
 'cylindernumber_indexed',
 'fuelsystem_indexed']

In [11]:
from pyspark.ml.feature import VectorAssembler

inp_features = ["fueltype_indexed", "aspiration_indexed", "doornumber_indexed", "carbody_indexed", "drivewheel_indexed", "enginelocation_indexed", "enginetype_indexed", "cylindernumber_indexed", "fuelsystem_indexed"]

# using all the generated numeric features
featureassembler=VectorAssembler(inputCols=inp_features,outputCol="Independent Features")
output=featureassembler.transform(df_indexed)

In [12]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|(9,[2,3,4,6],[1.0...|
|(9,[2,3,4,6],[1.0...|
|[0.0,0.0,1.0,1.0,...|
|           (9,[],[])|
| (9,[4,7],[2.0,2.0])|
| (9,[2,7],[1.0,2.0])|
|       (9,[7],[2.0])|
| (9,[3,7],[2.0,2.0])|
| (9,[1,7],[1.0,2.0])|
|[0.0,1.0,1.0,1.0,...|
| (9,[2,4],[1.0,1.0])|
|       (9,[4],[1.0])|
|(9,[2,4,7],[1.0,1...|
| (9,[4,7],[1.0,1.0])|
| (9,[4,7],[1.0,1.0])|
| (9,[4,7],[1.0,1.0])|
|(9,[2,4,7],[1.0,1...|
| (9,[4,7],[1.0,1.0])|
|[0.0,0.0,1.0,1.0,...|
|(9,[2,3,8],[1.0,1...|
+--------------------+
only showing top 20 rows



In [15]:
dataset = output.select("Independent Features","price")
dataset.show()

+--------------------+---------+
|Independent Features|    price|
+--------------------+---------+
|(9,[2,3,4,6],[1.0...|  13495.0|
|(9,[2,3,4,6],[1.0...|  16500.0|
|[0.0,0.0,1.0,1.0,...|  16500.0|
|           (9,[],[])|  13950.0|
| (9,[4,7],[2.0,2.0])|  17450.0|
| (9,[2,7],[1.0,2.0])|  15250.0|
|       (9,[7],[2.0])|  17710.0|
| (9,[3,7],[2.0,2.0])|  18920.0|
| (9,[1,7],[1.0,2.0])|  23875.0|
|[0.0,1.0,1.0,1.0,...|17859.167|
| (9,[2,4],[1.0,1.0])|  16430.0|
|       (9,[4],[1.0])|  16925.0|
|(9,[2,4,7],[1.0,1...|  20970.0|
| (9,[4,7],[1.0,1.0])|  21105.0|
| (9,[4,7],[1.0,1.0])|  24565.0|
| (9,[4,7],[1.0,1.0])|  30760.0|
|(9,[2,4,7],[1.0,1...|  41315.0|
| (9,[4,7],[1.0,1.0])|  36880.0|
|[0.0,0.0,1.0,1.0,...|   5151.0|
|(9,[2,3,8],[1.0,1...|   6295.0|
+--------------------+---------+
only showing top 20 rows



In [16]:
from pyspark.ml.regression import LinearRegression

# train test split in 80-20
train_data,test_data = dataset.randomSplit([0.80,0.20]
                                                )
lin_reg=LinearRegression(featuresCol='Independent Features', labelCol='price')
lin_reg=lin_reg.fit(train_data)

In [18]:
print(lin_reg.coefficients)
print(lin_reg.intercept)

[2191.277650748367,4294.730185935873,-1310.6583685301268,34.22110374447005,3437.212005294794,18751.716572220543,-380.8578132636653,3951.607326597133,-1387.5514924267177]
10959.206414575678


In [19]:
pred_results=lin_reg.evaluate(test_data)
pred_results.predictions.show()

+--------------------+-------+------------------+
|Independent Features|  price|        prediction|
+--------------------+-------+------------------+
|           (9,[],[])| 8195.0|10959.206414575678|
|           (9,[],[])| 8948.0|10959.206414575678|
|(9,[0,1,8],[1.0,1...|13845.0|14670.111266406482|
| (9,[0,8],[1.0,2.0])| 7898.0|10375.381080470608|
| (9,[0,8],[1.0,2.0])|10795.0|10375.381080470608|
|(9,[1,2,3],[1.0,1...| 7957.0|13977.499335725894|
|(9,[1,2,3,6],[1.0...|18150.0|12834.925895934899|
|(9,[1,2,3,8],[1.0...| 7957.0| 8427.293366019023|
| (9,[1,7],[1.0,2.0])|23875.0|23157.151253705815|
| (9,[2,3],[1.0,1.0])|11850.0| 9682.769149790021|
| (9,[2,3],[1.0,4.0])|11595.0|  9785.43246102343|
|(9,[2,3,4],[1.0,1...|22018.0|13119.981155084815|
|(9,[2,3,8],[1.0,1...| 5572.0| 8295.217657363304|
|(9,[2,3,8],[1.0,1...| 6189.0| 8295.217657363304|
|(9,[2,3,8],[1.0,1...| 6529.0| 5520.114672509868|
|(9,[2,3,8],[1.0,1...| 9095.0| 5520.114672509868|
| (9,[2,4],[1.0,1.0])|16430.0|13085.760051340345|


In [20]:
# Performance Metrics

print(pred_results.r2)
print(pred_results.meanAbsoluteError)
print(pred_results.meanSquaredError)

0.6751524520284071
3012.4215127284815
16918943.737693876
